<a href="https://colab.research.google.com/github/ankitshripalsingh/Operations-Research/blob/OR/Banking%20Case%20Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Bank marketing**

##  Problem Statement

You work as a Business Analytics Consultant at the Bank of Corporate. The bank provides financial services/products such as savings accounts, current accounts, debit cards, etc. to its customers. In order to increase its overall revenue, the bank conducts various marketing campaigns for its financial products such as credit cards, term deposits, loans, etc. These campaigns are intended for the bank’s existing customers. However, the marketing campaigns need to be cost-efficient so that the bank not only increases their overall revenues but also the total profit.

In 2019, the bank conducted a telemarketing campaign for one of its financial products ‘Term Deposits’ to help foster long-term relationships with the existing customers. The dataset (provided below) contains the information about all the customers who were contacted during this year to open term deposit accounts.


What are term deposits?

Term deposits, also called fixed deposits, are the cash investments made for a specific time period ranging from 1 month to 5 years for predetermined fixed interest rates. The fixed interest rates offered for term deposits are higher than the regular interest rates for savings accounts. The customers receive the total amount (investment plus the interest) at the end of the maturity period. Also, the money can only be withdrawn at the end of the maturity period. Withdrawing money before that will result in an added penalty associated, and the customer will not receive any interest returns. This kind of investment deposits provides the required funds to the bank for lending money to the corporates or individuals at a higher interest rate than what is paid to the customer.

**Business objectives**

Your aim is to identify the target customer segments for the term deposits from the pool of the bank’s existing customers.


**Optimisation**
For its future marketing campaign, the bank has allocated a budget of ₹1,50,000 and has also decided to segment the customers based on their marital status and educational background. Also, notably, the cost incurred by the bank for a one-minute call to any customer is ₹10. Considering all these factors, you, as a consultant, need to provide the analysis to the bank regarding the number of calls to be made to each customer segment ('Customer segment' is explained below) such that the total number of customers opening the term deposit account is maximised.

**Customer Segment**

In the data set, the customers are segregated based on their marital status and educational background. 'Marital status' and 'educational background' has three categories as follows.

| Marital Status | Educational Background |
| :---- | :---- |
Single|Bachelors
Married|Masters
Divorced|Doctorate


Each combination of marital status and educational background is considered as a customer segment. 



*Examples:* ‘Single - Bachelors’ is considered as one segment. ‘Single - Masters’ is considered as one segment. Similarly, ‘Married - Masters’ is considered as one segment, ‘Married - Doctorates’ is considered as one segment and so on.



*The Conditions of the Bank*:

- The bank is concerned about the overall customer diversification. It wants to ensure that it reaches out to all the customer segments. For this, it has provided you with the following information to include in your analysis:

    - From each customer segment (‘customer segment’ as explained above), at least 50 customers need to be contacted.

    - The total number of calls made to each customer category should meet the minimum number of calls (provided in the data file). 
    - The total number of conversions for the customer categories should match a minimum number.
 
*Main Objective of the optimisation problem*:

Within the conditions given above, you need to estimate the number of calls that have to be made for each customer segment such that the total estimated no of converted calls for the future marketing campaign is maximised.


##  Mathematical formulation / Pyomo components

---

**Sets:** 

The indexes for the given problem are, <br>
- Marital status, $m \in marital\_status$
- Degree status, $d \in degree$

---

**Parameters:** 

- $\text{duration_nconverted}_{m,d} \text{ - Avg. call duration for non-converted calls}$ <br>
- $\text{duration_converted}_{m,d} \text{ - Avg. call duration for converted calls}$ <br>
- $\text{conv_rate}_{m,d} \text{ - Conversion rate}$ <br>
- $\text{min_tcalls}_{m | d} \text{ - Minimum no. of calls by either marital status / degree}$ <br>
- $\text{min_ccalls}_{m | d} \text{ - Minimum no. of converted calls by either marital status / degree}$ <br>


---

**Decision variables:**
- Total number of  calls (converted + non-converted) to be made for each customer segment <br>

> ${total\_calls}_{m,d} \,\,\,\,\,\,\, \text{where} \ m \in marital\_status, d \in degree $<br>


---

**Objective:** <br>

To maximize conversions for each combination of marital status and degree



\begin{align}
\textrm{max} \sum \limits _{m,d} \text{total_calls}_{m,d}* \text{conv_rate}_{m,d}
\end{align}
where $m \in marital\_status$ and $d \in degree$

---

**Constraints:**



*   At-least 50 customers need to be contacted from each segment


>- $\text{total_calls}_{m,d}\geq\text{50}\ \ \ \ \forall {d \in degree, m \in marital\_status}$


*   Minimum number of total calls for each marital status


>- $\sum \limits _{d} \text{total_calls}_{m,d} \geq\text{min_tcalls}_{m}$


*   Minimum number of total calls for each degree


>- $\sum \limits _{m} \text{total_calls}_{m,d} \geq\text{min_tcalls}_{d}$


*   Minimum number of converted calls for each marital status


>- $\sum \limits _{d} (\text{total_calls}_{m,d}* \text{conv_rate}_{m,d})  \geq\text{min_tcalls}_{m}$


*   Minimum number of converted calls for each degree


>- $\sum \limits _{m} (\text{total_calls}_{m,d}* \text{conv_rate}_{m,d})  \geq\text{min_tcalls}_{d}$


*   Budget Constraint

\begin{align}
\sum \limits _{m,d} \Bigg[\text{total_calls}_{m,d}* \text{conv_rate}_{m,d}* \frac{\text{duration_converted}_{m,d}}{60}* \text{cost_per_min}\Bigg] & +\Bigg[\text{total_calls}_{m,d}* \text{(1- conv_rate}_{m,d})* \frac{\text{duration_nconverted}_{m,d}}{60}* \text{cost_per_min}\Bigg]  \leq\text{total_budget}
\end{align}

----


In [1]:
# Only for colab users
!pip install -q pyomo
!apt-get install -y -qq coinor-cbc
!apt-get install -y -qq glpk-utils

     |████████████████████████████████| 9.6 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

### Step1:
<b> Import Pyomo Environment </b>

In [4]:
from pyomo.environ import *
import pandas as pd

### Step2:
<b>Specify / import data</b>

In [9]:
# Reading the data from the Excel workbook - Bank_marketing.xlsx
InputData = "https://github.com/ankitshripalsingh/Operations-Research/raw/OR/Bank_marketing_input.xlsx"

#Read the data from Campaign_Data sheet
data = pd.read_excel(InputData, sheet_name='Campaign_Data')

#Read the data from Call criterias sheet
criteria = pd.read_excel (InputData, sheet_name='Call criterias')

#Total budget for marketing
total_budget=150000

#cost per 10 mins of a call
cost_per_min=10

In [10]:
#Printing the dataframe 
data

,Marital Status,Degree,Avg. Call Duration (Not Converted),Avg. Call Duration (Converted),Conversion Rate
0,Married,Bachelors,220.73,626.00,0.0755
1,Married,Masters,217.85,565.00,0.0929
2,Married,Doctorates,222.91,485.00,0.1278
3,Single,Bachelors,217.49,708.03,0.1062
4,Single,Masters,216.28,544.57,0.1271
5,Single,Doctorates,226.55,494.00,0.1812
6,Divorced,Bachelors,222.65,589.86,0.1367
7,Divorced,Masters,213.91,607.00,0.1030
8,Divorced,Doctorates,218.11,567.00,0.1383


In [11]:
#Printing the dataframe 
criteria

,Customer Segment,Minimum conversion,Minimum number of calls
0,Bachelors,120,400
1,Masters,120,500
2,Doctorates,120,600
3,Married,150,600
4,Single,150,300
5,Divorced,100,350


**Sets:** 

The indexes for the given problem are, <br>
- Marital status, $m \in marital\_status$
- Degree status, $d \in degree$

In [13]:
# Extracting the unique values of marital status and the educational degree which will act as indexes for
# the decision variables and the parameters
M_status = data['Marital Status'].unique()
Degree = data['Degree'].unique()
print(M_status, Degree)

['Married' 'Single' 'Divorced'] ['Bachelors' 'Masters' 'Doctorates']


**Parameters:** 

- $\text{duration_nconverted}_{m,d} \text{ - Avg. call duration for non-converted calls}$ <br>
- $\text{duration_converted}_{m,d} \text{ - Avg. call duration for converted calls}$ <br>
- $\text{conv_rate}_{m,d} \text{ - Conversion rate}$ <br>
- $\text{min_tcalls}_{m | d} \text{ - Minimum no. of calls by either marital status / degree}$ <br>
- $\text{min_ccalls}_{m | d} \text{ - Minimum no. of converted calls by either marital status / degree}$ <br>


In [16]:
# Creating the required data structures for the parameters with [Marital status, Degree] as indexes

# Duration of calls for the customers not converted
duration_nconverted = data.set_index(['Marital Status', 'Degree'])['Avg. Call Duration (Not Converted)'].to_dict()

# Duration of calls for the customers converted
duration_converted = data.set_index(['Marital Status', 'Degree'])['Avg. Call Duration (Converted)'].to_dict()

# Conversion rate
conv_rate = data.set_index(['Marital Status', 'Degree'])['Conversion Rate'].to_dict()

# Minimum no. of  total calls
min_tcalls = criteria.set_index(['Customer Segment'])['Minimum number of calls'].to_dict()

# Minimum no. of converted calls
min_ccalls = criteria.set_index(['Customer Segment'])['Minimum conversion'].to_dict()

In [17]:
duration_nconverted

{('Divorced', 'Bachelors'): 222.65,
 ('Divorced', 'Doctorates'): 218.11,
 ('Divorced', 'Masters'): 213.91,
 ('Married', 'Bachelors'): 220.73,
 ('Married', 'Doctorates'): 222.91,
 ('Married', 'Masters'): 217.85,
 ('Single', 'Bachelors'): 217.49,
 ('Single', 'Doctorates'): 226.55,
 ('Single', 'Masters'): 216.28}

In [18]:
duration_converted

{('Divorced', 'Bachelors'): 589.86,
 ('Divorced', 'Doctorates'): 567.0,
 ('Divorced', 'Masters'): 607.0,
 ('Married', 'Bachelors'): 626.0,
 ('Married', 'Doctorates'): 485.0,
 ('Married', 'Masters'): 565.0,
 ('Single', 'Bachelors'): 708.03,
 ('Single', 'Doctorates'): 494.0,
 ('Single', 'Masters'): 544.57}

In [19]:
conv_rate

{('Divorced', 'Bachelors'): 0.1367,
 ('Divorced', 'Doctorates'): 0.1383,
 ('Divorced', 'Masters'): 0.103,
 ('Married', 'Bachelors'): 0.0755,
 ('Married', 'Doctorates'): 0.1278,
 ('Married', 'Masters'): 0.0929,
 ('Single', 'Bachelors'): 0.1062,
 ('Single', 'Doctorates'): 0.1812,
 ('Single', 'Masters'): 0.1271}

In [20]:
min_tcalls

{'Bachelors': 400,
 'Divorced': 350,
 'Doctorates': 600,
 'Married': 600,
 'Masters': 500,
 'Single': 300}

In [21]:
min_ccalls

{'Bachelors': 120,
 'Divorced': 100,
 'Doctorates': 120,
 'Married': 150,
 'Masters': 120,
 'Single': 150}

### Step3:
<b> Create Model Object</b> <br>

In [30]:
#Instantiating a model
model = ConcreteModel()

### Step4 

<b>Define Decision Variable</b>:

- Total number of  calls (converted + non-converted) to be made for each customer segment <br>

> ${total\_calls}_{m,d} \,\,\,\,\,\,\, \text{where} \ m \in marital\_status, d \in degree $<br>



In [31]:
#Total number of  calls
model.total_calls = Var(M_status, Degree, within = PositiveIntegers)
model.total_calls.pprint()

total_calls : Size=9, Index=total_calls_index
    Key                        : Lower : Value : Upper : Fixed : Stale : Domain
     ('Divorced', 'Bachelors') :     1 :  None :  None : False :  True : PositiveIntegers
    ('Divorced', 'Doctorates') :     1 :  None :  None : False :  True : PositiveIntegers
       ('Divorced', 'Masters') :     1 :  None :  None : False :  True : PositiveIntegers
      ('Married', 'Bachelors') :     1 :  None :  None : False :  True : PositiveIntegers
     ('Married', 'Doctorates') :     1 :  None :  None : False :  True : PositiveIntegers
        ('Married', 'Masters') :     1 :  None :  None : False :  True : PositiveIntegers
       ('Single', 'Bachelors') :     1 :  None :  None : False :  True : PositiveIntegers
      ('Single', 'Doctorates') :     1 :  None :  None : False :  True : PositiveIntegers
         ('Single', 'Masters') :     1 :  None :  None : False :  True : PositiveIntegers


### Step5:
<b>Define Objective</b>


$\textrm{max} \sum \limits _{m,d} \text{total_calls}_{m,d}* \text{conv_rate}_{m,d}$


In [32]:
# Defining the objective rule
def obj_rule(model):
  return(sum(model.total_calls[m,d]*conv_rate[m,d] for m in M_status for d in Degree))

In [33]:
# Maximize the reach i.e, converted calls
model.value = Objective(rule = obj_rule, sense = maximize)

### Step6:
<b>Define Constraints</b>

*   At-least 50 customers need to be contacted from each segment


>- $\text{total_calls}_{m,d}\geq\text{50}\ \ \ \ \forall {d \in degree, m \in marital\_status}$




In [34]:
#At least 50 customers need to be contacted from each customer segment.
def min_calls_per_segment(model, m, d):
  return(model.total_calls[m,d] >= 50)

model.min_calls = Constraint(M_status, Degree, rule = min_calls_per_segment)

In [35]:
# Print the dictionary of minimum number of total calls to be made for each segment
min_tcalls

{'Bachelors': 400,
 'Divorced': 350,
 'Doctorates': 600,
 'Married': 600,
 'Masters': 500,
 'Single': 300}

*   Minimum number of total calls for each marital status


>- $\sum \limits _{d} \text{total_calls}_{m,d} \geq\text{min_tcalls}_{m}$


*   Minimum number of total calls for each degree


>- $\sum \limits _{m} \text{total_calls}_{m,d} \geq\text{min_tcalls}_{d}$



In [36]:
#The total number of calls made to each customer segment should meet the minimum number (based on marital status)
def min_tcalls_marital_status(model, m):
  return(sum(model.total_calls[m,d] for d in Degree) >= min_tcalls[m])

model.min_tcalls_mstatus = Constraint(M_status, rule = min_tcalls_marital_status)

#The total number of calls made to each customer segment should meet the minimum number (based on the degree)
def min_tcalls_degree(model, d):
  return(sum(model.total_calls[m,d] for m in M_status) >= min_tcalls[d])

model.min_tcalls_deg = Constraint(Degree, rule = min_tcalls_degree)

In [37]:
# Minimum number of converted calls to be made for each segment
min_ccalls

{'Bachelors': 120,
 'Divorced': 100,
 'Doctorates': 120,
 'Married': 150,
 'Masters': 120,
 'Single': 150}


*   Minimum number of converted calls for each marital status


>- $\sum \limits _{d} (\text{total_calls}_{m,d}* \text{conv_rate}_{m,d})  \geq\text{min_tcalls}_{m}$

In [39]:
#The total number of conversions/converted calls of the customer segments should match a minimum number(based on marital status)
def min_ccalls_marital_status(model, m):
  return(sum(conv_rate[m,d]*model.total_calls[m,d] for d in Degree) >= min_tcalls[m])

model.min_ccalls_mstatus = Constraint(M_status, rule = min_ccalls_marital_status)

*   Minimum number of converted calls for each degree


>- $\sum \limits _{m} (\text{total_calls}_{m,d}* \text{conv_rate}_{m,d})  \geq\text{min_tcalls}_{d}$


In [40]:
#The total number of conversions/converted calls of the customer segments should match a minimum number (based on degree)
def min_ccalls_degree(model, d):
  return(sum(conv_rate[m,d]*model.total_calls[m,d] for m in M_status) >= min_tcalls[d])

model.min_ccalls_deg = Constraint(Degree, rule = min_ccalls_degree)

*   Budget Constraint

\begin{align}
\sum \limits _{m,d} \Bigg[\text{total_calls}_{m,d}* \text{conv_rate}_{m,d}* \frac{\text{duration_converted}_{m,d}}{60}* \text{cost_per_min}\Bigg] & +\Bigg[\text{total_calls}_{m,d}* \text{(1- conv_rate}_{m,d})* \frac{\text{duration_nconverted}_{m,d}}{60}* \text{cost_per_min}\Bigg]  \leq\text{total_budget}
\end{align}

In [41]:
#Budget constraint
def maximum_budget(model):
  return(sum((duration_converted[m,d]/60)*cost_per_min*model.total_calls[m,d]*conv_rate[m,d] + (duration_nconverted[m,d]/60)*cost_per_min*model.total_calls[m,d]*(1-conv_rate[m,d]) for m in M_status for d in Degree) <= total_budget)

model.max_budget = Constraint(rule = maximum_budget)

### Step7:
<b>Create solver & solve model </b><br>

In [42]:
#Invoking the solver
result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 20
  Number of variables: 10
  Number of nonzeros: 46
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.017731666564941406


### Step8:
<b>Display Results </b>

In [43]:
model.pprint()

10 Set Declarations
    min_calls_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                              : Size : Members
        None :     2 : min_calls_index_0*min_calls_index_1 :    9 : {('Divorced', 'Bachelors'), ('Divorced', 'Doctorates'), ('Divorced', 'Masters'), ('Married', 'Bachelors'), ('Married', 'Doctorates'), ('Married', 'Masters'), ('Single', 'Bachelors'), ('Single', 'Doctorates'), ('Single', 'Masters')}
    min_calls_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Divorced', 'Married', 'Single'}
    min_calls_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Bachelors', 'Doctorates', 'Masters'}
    min_ccalls_deg_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Bachelors', 'Doctorates', 'Masters'}
 

In [ ]:
#Total number of calls to be made


In [ ]:
# Creating a dataframe for the output data

#Creating an empty list


# For every iteration, the code below will create a list containing 5 values - marital status, degree, number of calls for each segment
#number of converted calls and estimated cost for those calls

        
        #Number of calls to be made for each segment
       
        
        #converted calls
       
        
        #estimated cost
        
        





In [ ]:
#Converting a 2D list into a dataframe



In [ ]:
# Printing the data frame